# Intento de MLOPS

Estamos en pleno HotSale en la empresa en la que trabajo así que no tuve mucho tiempo para debuggear esto y documentarlo, pero dejo la estructura, aunque no corre (pero por pequeños detalles) 

In [1]:
#Instalaciones
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.0 MB/s eta 0:00:00


In [24]:
#Paquetería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import numpy as np
import re
import unidecode
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from joblib import load
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Drops

In [15]:
def first_drop(raw_data):
  raw_data.drop(['subtitle','timestamp',
                'price_currency','age_in_years', 'department_type','orientation',
                'floor_situated','disposition','main_name','link', 'price', 'price_mod', 'price_square_meter',
                'since_period', 'since_value', 'attributes','id'], axis = 1,
                inplace = True)
  
  return raw_data

# Limpieza de Texto

In [4]:
def addressPreprocessing(raw_data, umbral=7):
  # Más facil manejar todo en minusculas, sin acentos y controlando los espacios, al no ser
  # campos uniformes, tenemos una columna muy sucia, parece que cada quien que publica su 
  # departamento debe ingresar a mano la direccion.
  raw_data['location'] = (raw_data['location'].apply(lambda x: unidecode. #unidecode sirve para limpiar los acentos
                                                    unidecode(x.lower(). #con lower nos hacemos la vida más fácil al trabajar en minusculas
                                                              strip()))) #con strip controlamos los espacios antes y después

  raw_data['colonia'] = (raw_data['location']
                        .apply(lambda x: 
                                re.search(',(.*?),', x).group(1).strip() # quédate con TODO (incluyendo espacios)entre las primeras dos comas
                                if re.search(',(.*?),', x) #checa que exista algo entre las dos primeras comas
                                else np.nan)) #si no, nan y nos clavamos bien

  raw_data['Estado'] = (raw_data['location']
                        .apply(lambda x: re.#utilizamos regex
                              search('(distrito federal|distrito federal|nuevo leon|nuevo león|baja california|distrito federal)', #sustituye por esto
                                      x).group() if re
                              .search('(distrito federal|cdmx|nuevo leon|nuevo leon|baja california|ciudad de mexico)', #busca estas formas de escribir el estaod
                                      x) else np.nan)) #si no está esa forma de escribir el estado, nos pone NaN y hacemos deep dive

  colonias = { # Todas las colonias que tenemos en el dataset (limitacion: es poco escalable)
      "\\bcondesa\\b":"condesa",
      "\\broma, monterrey\\b": "roma",
      "\\bprivada roma\\b": "roma",
      "\\broma sur, monterrey\\b": "roma",
      "\\broma privada\\b":"roma",
      "\\broma norte\\b": "roma norte",
      "\\broma sur\\b": "roma_sur",
      "\\bqueretaro\\b": "queretaro",
      "\\bhermosillo\\b": "hermosillo",
      "\\bcuauhtemoc\\b": None,
      "\\bnarvarte\\b": "narvarte",
      "\\bdel valle\\b": "del valle",
      "\\bbaja california\\b":"baja california",
      "\\bbenito juarez\\b":None
  }

  raw_data['colonia'] = raw_data['location']

  for old_word, new_word in colonias.items(): # Buscamos si la colonia que identificamos existe en la direccion
      mask = raw_data['colonia'].str.contains(old_word, case=False, na=False)
      raw_data.loc[mask, 'colonia'] = new_word

  value_counts = raw_data['colonia'].value_counts()

  to_replace = value_counts[value_counts < umbral].index # Cambiamos aquellas colonias que aparecen menos que un umbral por "otra"

  raw_data.loc[raw_data['colonia'].isin(to_replace), 'colonia'] = 'otra'

  raw_data.loc[raw_data['Estado'] == 'baja california', 'colonia'] = 'baja california'

  encoder = OrdinalEncoder() # Nuestra columna de estados es una columna categórica
  raw_data['Estado_encoded'] = encoder.fit_transform(raw_data[['Estado']])

  #Nuestro problem domain y dataset nos da 5 clústers en los que podría caer
  kmeans = KMeans(n_clusters=5) #Las 5 categorías que tenemos: baja california, roma nuevo leon, roma N, roma S, Otra
  labels = kmeans.fit_predict(raw_data[['lat', 'lon', 'Estado_encoded', 'final_price']])

  raw_data['Colonia_KMeans'] = labels

  #Agregamos la moda como etiqueta de Colonia
  label_colonia = {}
  for label in set(labels):
      colonia = raw_data.loc[raw_data['Colonia_KMeans'] == label, 'colonia'].mode().iloc[0] # Hacemos la imputación de la colonia por la moda (al ser categorico)
      label_colonia[label] = colonia

  raw_data.loc[raw_data['colonia'].isnull(), 'colonia'] = (raw_data
                                                          .loc[raw_data['colonia'].isnull(),
                                                                'Colonia_KMeans'].map(label_colonia))
  
  return raw_data

# Drop 2

In [17]:
def second_drop(raw_data):
  raw_data.drop(['location', 'address',
                'since','Estado_encoded', 'Colonia_KMeans', 'num_floors', 'monthly_fee',
                'apartments_per_floor', 'description', 'cellars'], 
                axis = 1, inplace = True)
  
  raw_data.dropna(inplace=True)

  return raw_data

# Encoding


In [6]:
def encoding(raw_data):
  colonia_encoded = pd.get_dummies(raw_data['colonia'])
  estado_econded = pd.get_dummies(raw_data['Estado'])

  clean_data = pd.concat([colonia_encoded, estado_econded, raw_data], axis=1)
  clean_data.drop(['colonia', 'Estado', 'vendor'], inplace = True, axis=1)

  return clean_data

# Data Preprocessing / Splitting

In [7]:
def split_scale(clean_data, test_size=0.2):
  X = clean_data.drop('final_price', axis=1)
  y = clean_data['final_price']

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

  scaler = StandardScaler()

  scaler.fit(x_train)

  x_train = scaler.transform(x_train)
  x_test = scaler.transform(x_test)

  return x_train, x_test, y_train, y_test

# Models, metrics and results

In [8]:
def MAPE(y_test, y_pred): 
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [9]:
def xGBoost_PriceModel(x_train, y_train, x_test = None, y_test = None, feature_ranking = True, path_to_weights = None):
  xg = GradientBoostingRegressor(learning_rate= 0.1, n_estimators= 200)
  xg.fit(x_train, y_train)

  if x_test and y_test:
    y_pred = xg.predict(x_test)
    print(f"xGBoost RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")
    print(f"R2: {xGb.score(x_test,y_test)}")
    print(f'MAPE: {MAPE(y_test,y_pred)}')

  if feature_ranking:
    importances = xg.feature_importances_

    feature_names = X.columns

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })

    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    importance_df = importance_df.reset_index(drop=True)

    print("Feature ranking:")
    for i in range(importance_df.shape[0]):
        print(f"Feature {importance_df.loc[i, 'Feature']}, Importance: {importance_df.loc[i, 'Importance']}")

    if path_to_weights:
      dump(xg, os.path.join(path_to_weights,'xg_model.joblib'))

    return xg

In [10]:
def rFBoost_PriceModel(x_train, y_train, x_test = None, y_test = None, feature_ranking = True, path_to_weights = None):
  rf = RandomForestRegressor(max_depth= None, min_samples_leaf= 1, n_estimators= 150)
  rf.fit(x_train, y_train)

  if x_test and y_test:
    y_pred = rf.predict(x_test)
    print(f"Random Forest RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")
    print(f"R2: {forest_model.score(x_test,y_test)}")
    print(f'MAPE: {MAPE(y_test,y_pred)}')

  if feature_ranking:
    importances = rf.feature_importances_

    feature_names = X.columns

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })

    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    importance_df = importance_df.reset_index(drop=True)

    print("Feature ranking:")
    for i in range(importance_df.shape[0]):
        print(f"Feature {importance_df.loc[i, 'Feature']}, Importance: {importance_df.loc[i, 'Importance']}")

  if path_to_weights:
    dump(rf, os.path.join(path_to_weights,'rf_model.joblib'))

  return rf

In [11]:
def predictPrice(raw_data, model = 'xg', price='square_meter', umbral = 7,  test_size=0.2, path_to_weights=None):
  m2 = raw_data['m2']

  raw_data = first_drop(raw_data)

  raw_data = addressPreprocessing(raw_data, umbral)

  raw_data = second_drop(raw_data)

  clean_data = encoding(raw_data)

  if model == 'xg':
    if path_to_weights:
      model = load(path_to_weights)
    else:
      x_train, y_train,_,_ = split_scale(clean_data, test_size)
      model = xGBoost_PriceModel(x_train, y_train, x_test = None, y_test = None, feature_ranking = True)
  else:
    if path_to_weights:
      model = load(path_to_weights)
    else:
      x_train, y_train,_,_ = split_scale(clean_data, test_size)
      model = rFBoost_PriceModel(x_train, y_train, x_test = None, y_test = None, feature_ranking = True)
  
  X = clean_data.drop('final_price', axis=1)
  y_pred = model.predict(X)

  if price == "square_meter":
    return y_pred/m2
  else:
    return y_pred

In [27]:
#Carga de archivo
# Correr la siguiente instruccion solo si lo planean correr en Colab
#os.chdir("/content")
raw_data = pd.read_csv("./reto_precios.csv")

In [28]:
prediction = predictPrice(raw_data, model = 'xg', price='square_meter', umbral = 7,  test_size=0.2, path_to_weights=None)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: ignored